# [STEP3 - 解析1] ChatGPTによる解析方法の提案

STEP1, 2で適切なデータセットが得られたと仮定して検証を行いました。（※STEP1, 2のアルゴリズム調整が未対応のため）

今回は汎用的な解析アルゴリズムではなく、下記3つの解析カテゴリーについて解析方法を検討しました。

In [ ]:
# Pythonライブラリインストール
# ※Python 3.10.x　使用推奨
!python --version
!pip install python-dotenv
!pip install --upgrade openai
!pip install openai[datalib]

!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install plotly
!pip install scikit-learn
!pip install sqlalchemy

## 環境変数
supabase接続用URL,APIキーと、openai api接続用のAPIキーを設定します。
自身のopenaiアカウントからapi keyを取得してください。

https://platform.openai.com/account/api-keys

supabaseの情報は管理者にお尋ねください。

下記の例では、.envファイルに変数を書き込んで、JupiterNotebookで読み込む仕様で実装しております。

※.envファイルの作成が困難、.envファイルから値を読み込めない場合、
　os.getenv("◯◯")部分に変数値を直接書き込んでいただいても動作自体には問題ありません。

In [30]:
# 環境変数
import os
from dotenv import load_dotenv

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
import urllib.parse
from IPython.display import display


load_dotenv()

# supabase接続用変数
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")

# OPENAI API KEY
openai_api_key = os.getenv("OPENAI_API_KEY")


# Connect to the database
connection_config = {
    'user': db_user,
    'password': urllib.parse.quote_plus(db_pass),
    'host': db_host,
    'port': db_port, 
    'database': db_name
}
engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**connection_config))


print('環境変数読み込み完了')

環境変数読み込み完了


# 処理実行
## 解析1. 大まかな脳領域からの投射関係を出力

Frontal Pole (ID:184)から他の脳部位への投射関係について解析しました。
解析対象は、脳の階層構造において、子ノードを持たない脳部位に絞っております。


In [31]:
# 投射元のstructure id
# 入力例:
#    Frontal Pole (ID:184)
#    Primary motor area (ID:985)
#    Secondary motor area (ID:993)
input_structure_id = 184
# 正規化投射量(normalized projection volume)の多い順に{rank_num}項目だけ書き出し
rank_num = 20


# 子要素を全て書き出し
sql ="""
SELECT
    id,
    name,
    acronym,
    "st-level",
    "parent-structure-id"
FROM
    structures;"""
with engine.begin() as conn:
    query = text(sql)
    df_structures = pd.read_sql_query(query, conn)

structure_ids = [input_structure_id]
terminal_structure_ids = df_structures.loc[~df_structures['id'].isin(df_structures['parent-structure-id'])]['id']

def get_children(parent_id):
    return df_structures[df_structures["parent-structure-id"] == parent_id]

def build_hierarchy(parent_id, level=0):
    children = get_children(parent_id)
    hierarchy = {}
    for _, child in children.iterrows():
        child_id = child["id"]
        hierarchy[child_id] = {"level": level, "acronym": child['acronym'], "children": build_hierarchy(child_id, level + 1)}
        structure_ids.append(child_id)
    return hierarchy

hierarchy = build_hierarchy(input_structure_id)


# 投射情報
sql ="""
SELECT
    p.id,
    p."structure-id" AS "projected-structure-id",
    s.name,
    s.acronym,
    p."normalized-projection-volume",
    sp."structure-id"
FROM projections AS p
    INNER JOIN specimens AS sp ON p."experiment-id" = sp."experiment-id"
    INNER JOIN structures AS s ON p."structure-id" = s.id
WHERE
    sp."structure-id" IN :structure_ids
    AND p."is-injection" = false
    AND s."st-level" > 5;
"""
with engine.begin() as conn:
    query = text(sql)
    df = pd.read_sql_query(query, conn, params={"structure_ids": tuple(structure_ids)})

# Filter structures which have no child node
df=df[df['projected-structure-id'].isin(terminal_structure_ids)]
df=df[~df['projected-structure-id'].isin(structure_ids)]

# display(df.sort_values(by="normalized-projection-volume", ascending=False))

# Calculate the average volume for each id
df_groupby = df.copy()
df_groupby.loc[:, 'label'] = df_groupby['acronym'] + "(" + df_groupby['name'] + ")"

average_volume = df_groupby.groupby('label')['normalized-projection-volume'].mean()

# Sort the average volume
sorted_average_volume = average_volume.sort_values(ascending=False)


display(sorted_average_volume.head(rank_num))


# export only ids
average_volume = df_groupby.groupby('projected-structure-id')['normalized-projection-volume'].mean()
sorted_average_volume = average_volume.sort_values(ascending=False)
ids = sorted_average_volume.head(rank_num)
r = ids.index[:rank_num].tolist()

print("投射元 structures ids")
print(structure_ids)

print("投射先 structures ids")
print(r)

label
CP(Caudoputamen)                                          2.440266
MOs5(Secondary motor area, layer 5)                       0.961629
MOs2/3(Secondary motor area, layer 2/3)                   0.715561
MOs1(Secondary motor area, layer 1)                       0.510339
MOs6a(Secondary motor area, layer 6a)                     0.338765
VAL(Ventral anterior-lateral complex of the thalamus)     0.311510
AId5(Agranular insular area, dorsal part, layer 5)        0.301772
VM(Ventral medial nucleus of the thalamus)                0.287174
AId2/3(Agranular insular area, dorsal part, layer 2/3)    0.217278
PO(Posterior complex of the thalamus)                     0.169447
AId6a(Agranular insular area, dorsal part, layer 6a)      0.156987
MOp1(Primary motor area, Layer 1)                         0.124162
MOp2/3(Primary motor area, Layer 2/3)                     0.120033
MOp6a(Primary motor area, Layer 6a)                       0.111396
AId1(Agranular insular area, dorsal part, layer 1)      

投射元 structures ids
[184, 526322264, 526157192, 526157196, 667, 68]
投射先 structures ids
[672, 767, 962, 656, 1021, 629, 1101, 685, 328, 1020, 783, 320, 943, 844, 996, 262, 599, 630, 907, 440]
